In [95]:
from PIL import Image
import numpy as np
import random
import sys
import os
import shutil 

import matplotlib.pyplot as plt
from Crypto.Cipher import ChaCha20
from Crypto.Random import get_random_bytes
from Crypto.Protocol.KDF import HKDF
from Crypto.Hash import SHA256, HMAC

In [96]:
#importing the image
def import_image(image_path):
    image = Image.open(image_path)
    image = image.convert("RGB")
    image = image.resize((256, 256))
    return image

# **Creating Unique Keys**

In [97]:
#Generating a unique id for all participants name the array ID
def generate_id(num_participants):
    ID = set()
    while len(ID) < num_participants:
        ID.add(random.randint(0, 1000))
    return list(ID)


# Creating unique key for each participant using HKDF and the master key r
def generate_key(r, ID, salt):
    key = []
    for i in ID:
        key.append(HKDF(r, 32, salt, SHA256, context=str(i).encode()))
    return key


# **Creating Pseudo-Shares**

In [98]:
#Generating a dictionary for the nonce values to each unique particapant id
def generate_nonce(ID):
    nonce = {}
    for i in ID:
        nonce[i] = get_random_bytes(12)
    return nonce

#ChaCha20 encryption for each participant using key, nonce and plaintext
def pseudo_share(key, nonce, plaintext):
    I = {}
    for i in range(len(key)):
        cipher = ChaCha20.new(key=key[i], nonce=nonce[i])
        I[i] = cipher.encrypt(plaintext)
    return I


In [ ]:
# HMAC Generation using SHA256
def generate_hmacs(pseudo_shares, num_participants):
    hmacs = []
    for i in range(num_participants):
        h = HMAC.new(pseudo_shares[i], digestmod=SHA256)
        hmacs.append(h.hexdigest())
    return hmacs


def generate_random_images(num_participants):
    random_images = []
    # Generate a random master key
    keys = [get_random_bytes(32)]
    for i in range(num_participants):
        key = get_random_bytes(32)
        keys.append(key)
        nonce = get_random_bytes(12)
        cipher = ChaCha20.new(key=key, nonce=nonce)
        random_images.append(cipher.encrypt(bytes(256 * 256 * 3)))
    return (random_images, keys)

def generate_random_images_with_keys(num_participants, keys):
    random_images = []
    for i in range(num_participants):
        nonce = get_random_bytes(12)
        cipher = ChaCha20.new(key=keys[i], nonce=nonce)
        random_images.append(cipher.encrypt(bytes(256 * 256 * 3)))
    return random_images

In [100]:
#secret Reconstruction using Lagrange Interpolation
def reconstruct_secret(shares, t):
    secret = 0
    for i in range(t):
        num = 1
        den = 1
        for j in range(t):
            if i != j:
                num *= -j
                den *= i - j
        secret += shares[i] * num // den
    return secret

In [105]:
IDs = generate_id(5)
salt = get_random_bytes(16)
r = get_random_bytes(16)
key = generate_key(r, IDs, salt)
nonce = generate_nonce(IDs)
image = import_image("lena.png")
image_array = np.array(image)
plaintext = "this is secret".encode()
pseudo_shares = pseudo_share(key, list(nonce.values()), plaintext)
print("Pseudo shares generated")
hmacs = generate_hmacs(pseudo_shares, 5)
random_images, coeffecients = generate_random_images(3)

# Convert byte images to numpy arrays
random_images = [np.frombuffer(img, dtype=np.uint8).reshape(image_array.shape) for img in random_images]

# Ensure image_array is uint8
image_array = image_array.astype(np.uint8)

# Perform XOR operation safely
public_share = image_array ^ random_images[0] ^ random_images[1] ^ random_images[2]

# Save the public share as an image 
public_share = Image.fromarray(public_share)
public_share.save("public_share.png")

# Just to check if the public share is correct
recovered_image = random_images[0] ^ random_images[1] ^ random_images[2] ^ public_share
recovered_image = Image.fromarray(recovered_image)
recovered_image.show()

Pseudo shares generated


In [113]:
def compute_polynomial_at_x(coefficients, x):
    result = 0
    for i in range(len(coefficients)):
        result += coefficients[i] * (x ** i)
    return result


def lagrange_interpolation(x_values, y_values):
    coefficients = [0] * len(x_values)
    for i in range(len(x_values)):
        numerator = 1
        denominator = 1
        for j in range(len(x_values)):
            if i != j:
                numerator *= -x_values[j]
                denominator *= x_values[i] - x_values[j]
        coefficients[i] = y_values[i] * numerator // denominator
    return coefficients

XY_values = []

for i in range(len(pseudo_shares)):
    pseudo_share_int = int.from_bytes(pseudo_shares[i], byteorder="big")  # Convert to integer
    coeffecients_ints = [int.from_bytes(c, byteorder="big") for c in coeffecients]  # Convert to integer
    XY_values.append((pseudo_share_int, compute_polynomial_at_x(coeffecients_ints, pseudo_share_int)))

print("XY values generated")

XY values generated


In [114]:
recovered_coefficients = lagrange_interpolation([x for x, y in XY_values], [y for x, y in XY_values])

In [117]:
# generate the random matrices using the same keys and nonces
# convert back recovered coefficients to bytes
recovered_coefficients = [int.to_bytes(c, 32, byteorder="big") for c in recovered_coefficients]
random_images, _ = generate_random_images_with_key(3, recovered_coefficients[1:])
random_images = [np.frombuffer(img, dtype=np.uint8).reshape(image_array.shape) for img in random_images]

# Perform XOR operation safely
recovered_image = random_images[0] ^ random_images[1] ^ random_images[2] ^ public_share

# Save the recovered image
recovered_image = Image.fromarray(recovered_image)
recovered_image.save("recovered_image.png")
recovered_image

OverflowError: int too big to convert